In [1]:
from flask import Flask
#from flask_cors import CORS
from flask import jsonify
from flask import request
import modelselection as ms
import pickle
import json
import preprocessing as prep
#import visualizedata as vd
#from flask_cors import CORS
from FraudDependencyParser import FraudDependencyParser
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

obj = FraudDependencyParser()

def model_selector():
    input_df=pd.read_csv(obj.inpath+'Input_dataset.csv')
    input_df= prep.preprocess(input_df)

    ## convert Categorical variables to Integer Variables
    df_fi = pd.concat([input_df, prep.convertcat_to_intVar(input_df)], axis=1)
    #split the dataset into train and Validation dataset
    trainX, testX, trainY, testY = prep.splitDataset(df_fi)
    
    model_performance = []

    model_performance.append(ms.FraudLogisticRegression(trainX, trainY, testX, testY))
    model_performance.append(ms.FraudDecisionTree(trainX, trainY, testX, testY))
    model_performance.append(ms.FraudXGB(trainX, trainY, testX, testY))
    model_performance.append(ms.FraudRandomForest(trainX, trainY, testX, testY))
    
    return model_performance


In [4]:
model_performance=model_selector()
print(model_performance)

[{'ModelName': 'LogisticRegression', 'precision': 0.8744239631336406, 'recall': 0.5532069970845481, 'f1_score': 0.6776785714285715, 'accuracy': 0.993077395106236, 'Avg_precision_recall': 0.7204457064076731, 'log_loss': 0.23909913815722084, 'mse': 0.006922604893763903, 'rmse': 0.08320219284227973}, {'ModelName': 'DecisionTree', 'precision': 0.82991452991453, 'recall': 0.7077259475218659, 'f1_score': 0.7639653815892998, 'accuracy': 0.9942471427475646, 'Avg_precision_recall': 0.7727333496440165, 'log_loss': 0.19869817593038197, 'mse': 0.0057528572524353765, 'rmse': 0.07584759226524845}, {'ModelName': 'XGBoostClassifier', 'precision': 0.8876306620209059, 'recall': 0.7427113702623906, 'f1_score': 0.8087301587301586, 'accuracy': 0.9953785380072102, 'Avg_precision_recall': 0.6626379829599761, 'log_loss': 0.15962063138006774, 'mse': 0.004621461992789753, 'rmse': 0.06798133562081399}, {'ModelName': 'RandomForest', 'precision': 0.8630252100840337, 'recall': 0.7485422740524781, 'f1_score': 0.8017

In [6]:
model_df = pd.DataFrame(model_performance)
model_df

,Avg_precision_recall,ModelName,accuracy,f1_score,log_loss,mse,precision,recall,rmse
0,0.720446,LogisticRegression,0.993077,0.677679,0.239099,0.006923,0.874424,0.553207,0.083202
1,0.772733,DecisionTree,0.994247,0.763965,0.198698,0.005753,0.829915,0.707726,0.075848
2,0.662638,XGBoostClassifier,0.995379,0.808730,0.159621,0.004621,0.887631,0.742711,0.067981
3,0.879813,RandomForest,0.995129,0.801717,0.168231,0.004871,0.863025,0.748542,0.069791


In [7]:
with open(obj.model_path+'model_performance.json', 'w') as f:
    json.dump(model_performance, f)

In [9]:
app = Flask(__name__)     
CORS(app) 

@app.route('/mfdm/selectmodel', methods=['POST'])
def configure():
    # Get the data from the POST request.
    data = request.get_json(force=True)
    
    return jsonify(model_performance)

app.run(host='127.0.0.1', port= 5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Feb/2020 15:40:03] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [20/Feb/2020 15:40:21] "GET /mfdm/selectmodel HTTP/1.1" 405 -
127.0.0.1 - - [20/Feb/2020 15:40:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [20/Feb/2020 15:40:44] "GET /mfdm/selectmodel HTTP/1.1" 405 -
